### Libraries

In [1]:
from typing import Union
import numpy as np
from qclib.gates.mcu import MCU
from qclib.gates.ldmcu import Ldmcu
from qiskit import (QuantumCircuit,
                    QuantumRegister,
                    transpile)
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit.quantum_info import state_fidelity, Statevector
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
import matplotlib.pyplot as plt

In [2]:
class Utilities:

    def __init__(self, operator, error):
        self.operator = operator
        self.error = error

    @staticmethod
    def pauli_matrices(string: str) -> Union[np.ndarray, None]:
        string_case = string.lower()
        match string_case:
            case "x":
                return np.array([
                    [0, 1],
                    [1, 0]
                ])
            case "y":
                return np.array([
                    [0, -1j],
                    [1j, 0]
                ])
            case "z":
                return np.array([
                    [1, 0],
                    [0, -1]
                ])
            case _:
                return None

    @staticmethod
    def transpose_conjugate(operator):
        return np.conjugate(np.transpose(operator))

    @staticmethod
    def pyramid_size(operator, error):
        mcu_dummy = MCU(operator, num_controls=100, error=error)
        # will be changed by mcu_dummy._get_n_base(x_dagger, error)
        return mcu_dummy._get_num_base_ctrl_qubits(operator, error)


In [3]:
def built_circuit(extra_qubits=0, pauli_string='x', error=0.1, approximated=True):
    
    pauli_matrix = Utilities.pauli_matrices(pauli_string)
    pauli_matrix_dagger = Utilities.transpose_conjugate(pauli_matrix)
    n_base = Utilities.pyramid_size(pauli_matrix_dagger, error)

    controls = QuantumRegister(n_base + extra_qubits, 'controls')
    target = QuantumRegister(1, 'target')
    circ = QuantumCircuit(controls, target)
    circ.x(list(range(len(controls))))
    
    if approximated:
        MCU.mcu(circ, pauli_matrix_dagger, controls, target, error)
    else:
        Ldmcu.ldmcu(circ, pauli_matrix_dagger, controls, target)
        
    # circ.measure_all()
    
    return circ

In [4]:
n_extra = 0
error = 0.1
matrix = 'x'

In [5]:
approx_circuit = built_circuit(n_extra, matrix, error)
print(approx_circuit)

            ┌───┐┌──────────────┐
controls_0: ┤ X ├┤0             ├
            ├───┤│              │
controls_1: ┤ X ├┤1             ├
            ├───┤│              │
controls_2: ┤ X ├┤2             ├
            ├───┤│              │
controls_3: ┤ X ├┤3 Ldmcuapprox ├
            ├───┤│              │
controls_4: ┤ X ├┤4             ├
            ├───┤│              │
controls_5: ┤ X ├┤5             ├
            └───┘│              │
    target: ─────┤6             ├
                 └──────────────┘


In [6]:
real_circuit = built_circuit(n_extra, matrix, error, False)
# print(real_circuit)

In [7]:
def state_vector(circuit):
    
    size = len(circuit)
    backend = GenericBackendV2(num_qubits=size)
    noise_model = NoiseModel.from_backend(backend)
    
    simulator = AerSimulator(noise_model=noise_model)
    transpiled_circuit = transpile(circuit, simulator)
    transpiled_circuit.save_statevector()
    
    job = simulator.run(transpiled_circuit)
    result = job.result()
    sv = result.get_statevector()
    # result = job.result().get_counts(0)
    # d_m = DensityMatrix(transpiled_circuit)
    return sv, size

In [8]:
rc_vector, n1 = state_vector(real_circuit)

In [9]:
t_array = np.zeros(2**n1)
t_array[-1] = 1
t_state_vector = Statevector(t_array)

In [10]:
ac_vector, n2 = state_vector(approx_circuit)

### State Fidelity

In [11]:
state_fidelity(t_state_vector, rc_vector)

0.9995214455068376

In [12]:
state_fidelity(t_state_vector, ac_vector)

0.9973564783459824